In [1]:
import pandas as pd
import requests
import json
import openpyxl
import configparser
cfg = configparser.ConfigParser()
cfg.read("secrets.ini")
my_key=cfg.get("default", "epa_apikey")
my_email=cfg.get('default','epa_email')

In [7]:
import requests
import urllib3
import ssl


class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [11]:
with (get_legacy_session()) as s: 
    response = s.get("https://aqs.epa.gov/data/api/signup?email=jensenl7@msu.edu")

In [12]:
response.status_code

200

In [13]:
response.json()

{'Header': [{'status': 'Success',
   'request_time': '2023-09-14T14:22:32-04:00',
   'url': 'https://aqs.epa.gov/data/api/signup?email=jensenl7@msu.edu'}],
 'Data': ['You should receive a registration confirmation email with a link for confirming your email shortly.']}

In [32]:
params = {
    "email" : my_email,
    "key"   : my_key,
    "state" : 26,   # Michigan code
    #"county": '081', # Kent County code
}

In [33]:
response = get_legacy_session().get("https://aqs.epa.gov/data/api/list/countiesByState", params=params)

In [34]:
response.status_code

200

In [58]:
params = {
    "email" : my_email,
    "key"   : my_key,
    "state" : 26,   # Michigan code
    #"county": '081', # Kent County code
    
}

In [86]:
response = get_legacy_session().get("https://aqs.epa.gov/data/api/list/parametersByClass?", params = {'email':my_email,'key':my_key,'pc':'AQI POLLUTANTS'})

In [88]:
param_codes_df = pd.DataFrame(response.json()['Data']).set_index('value_represented')

In [89]:
param_codes_df.head()

,code
value_represented,
Carbon monoxide,42101
Sulfur dioxide,42401
Nitrogen dioxide (NO2),42602
Ozone,44201
PM10 Total 0-10um STP,81102


In [90]:
param_codes_df.index

Index(['Carbon monoxide', 'Sulfur dioxide', 'Nitrogen dioxide (NO2)', 'Ozone',
       'PM10 Total 0-10um STP', 'PM2.5 - Local Conditions',
       'Acceptable PM2.5 AQI & Speciation Mass'],
      dtype='object', name='value_represented')

In [74]:
param_codes_df.loc['Volatile organic compounds'][0]

'VOC'

In [109]:
params = {
    "email" : my_email,
    "key"   : my_key,
    "state" : 26,   # Michigan code
    "county": '081', # Kent County code
    "bdate" : 20220101,
    "edate" : 20220601,
    "param" : param_codes_df.loc['Sulfur dioxide'][0],
    
}

In [110]:
response = get_legacy_session().get("https://aqs.epa.gov/data/api/transactionsSample/byCounty",params=params)

In [112]:
response  = get_legacy_session().get("https://aqs.epa.gov/data/api/dailyData/byCounty?",params=params)

In [113]:
pd.DataFrame(response.json()['Data']).columns

Index(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc',
       'latitude', 'longitude', 'datum', 'parameter', 'sample_duration_code',
       'sample_duration', 'pollutant_standard', 'date_local',
       'units_of_measure', 'event_type', 'observation_count',
       'observation_percent', 'validity_indicator', 'arithmetic_mean',
       'first_max_value', 'first_max_hour', 'aqi', 'method_code', 'method',
       'local_site_name', 'site_address', 'state', 'county', 'city',
       'cbsa_code', 'cbsa', 'date_of_last_change'],
      dtype='object')

In [114]:
pd.DataFrame(response.json()['Data']).to_excel('b.xlsx')